## Tabel of Contents:
* [1 Case 2. Pneumonia X-ray image analysis](#case-2)
* [2 Background](#case-background)
* [3 Data](#loading-data)
* [4 Exploratory Data Analysis And Preprocessing](#exploratory-analysis)
* [5 Models and Training](#models-trainning)
* [6 Conclusions](#cas-conclusion)

# Case 2. Pneumonia X-ray image analysis
 <a class="anchor" id="case-2"></a>
Team 14:<br>
* Awet Ghebreslassie
* Leevi Pelkonen
* Visa Soininen<br><br>
Last edited: 01.03.2020<br>
Neural Networks for Health Technology Applications<br>
[Helsinki Metropolia University of Applied Sciences](http://www.metropolia.fi/en/)<br>

# 2 Background  <a class="anchor" id="case-background"></a>

The aim of this notebook is to predict whether a person is sufferring from a lung-condition called Pneumonia. The early-phase or minor pneumonia can be hard to spot by looking at a chest-xray with the naked eye. Severe cases show clear fogginess in the lung area.

In this notebook we use Convolutional Neural Networks (CNN) to determine if a person has pneumonia.

# 3 Data  <a class="anchor" id="loading-data"></a>

The dataset is provided by Daniel Kermany, Kang Zhang and Michael Goldbaum from the University of California San Diego. The dataset contains chest x-ray images with persons suffering from pneunomia and healthy persons. The pictures are labeled and split into different sets if the lung appearing in the picture has pneumonia or not. Latest version of the dataset is published in 2018 and it is provided under the CC BY 4.0 -License.

To optimize the training and prediction time of the CNN, the pictures are downscaled from the original size.s) (Value 0 = < 50% diameter narrowing; Value 1 = > 50% diameter narrowing

In [ ]:
#imports
%pylab inline
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, roc_curve, confusion_matrix
from tensorflow.keras.metrics import SensitivityAtSpecificity

import os
import time

In [ ]:
# train_dir = "ChestXRay2017/chest_xray/train"
train_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/train'
test_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/test'
os.listdir(train_dir)

# 4 Exploratory Data Analysis And Preprocessing <a class="anchor" id="exploratory-analysis"></a>

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255, validation_split = 0.3)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
batch_size = 16
target_size = (158, 158)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = target_size,
    batch_size = batch_size,
    class_mode = 'binary',
    subset = 'training')
    
dev_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = target_size,
    batch_size = batch_size,
    class_mode = 'binary',
    subset = 'validation')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size= target_size,
    batch_size= batch_size, 
    class_mode='binary',
    shuffle= False)

test_labels = test_generator.classes
num_test_samples = len(test_generator.filenames)

In [ ]:
i = 0
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape', labels_batch.shape)
    i = i + 1
    if i > 5:
        break

In [ ]:
imshow(data_batch[0])
show()

# 5 Models and training  <a class="anchor" id="models-trainning"></a>

#### Functions

In [ ]:
def plot_history(history):
    '''
    Plots accuracy and loss from model history
    '''
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    sensitivity_specificity = history.history['sensitivity_at_specificity']
    val_sensitivity_specificity = history.history['val_sensitivity_at_specificity']
    epochs = range(len(acc))
    
    plot(epochs, acc, 'b', label = 'Training acc')
    plot(epochs, val_acc, 'r', label = 'Validation acc')
    title('Training and validation accuracy')
    grid()
    legend()

    figure()
    plot(epochs, loss, 'b', label = 'Training loss')
    plot(epochs, val_loss, 'r', label = 'Validation loss')
    title('Training and validation loss')
    grid()
    legend()
    
    figure()
    plot(epochs, sensitivity_specificity, 'b', label = 'Training sensitivity specificity')
    plot(epochs, val_sensitivity_specificity, 'r', label = 'Validation sensitivity specificity')
    title('Training and validation sensitivity specificity')
    grid()
    legend()

    show()

In [ ]:
def prediction_metrics(pred):
    print('Confusion matrix\n', confusion_matrix(test_labels, pred > 0.4).T)
    print('\nClassification report\n', classification_report(test_labels, pred > 0.4, target_names = ['Normal(0)', 'Pneumonia(1)']))
    
    fpr, tpr, thresholds = roc_curve(test_labels, pred, pos_label = 1)
    plot(fpr, tpr)
    plot([0, 1], [0, 1], 'r:')
    xlabel('False positive rate')
    ylabel('True positive rate')
    title('ROC curve')
    xlim([0, 1])
    ylim([0, 1])
    grid()

In [ ]:
metrics = [ SensitivityAtSpecificity(0.9, name= 'sensitivity_at_specificity'), 'acc']

### Model 1

In [ ]:
model_1 = models.Sequential()
model_1.add(layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (158, 158, 3)))
model_1.add(layers.MaxPooling2D((2, 2)))
model_1.add(layers.Conv2D(32, (3, 3), activation = 'relu'))
model_1.add(layers.MaxPooling2D((2, 2)))
model_1.add(layers.Conv2D(32, (3, 3), activation = 'relu'))
model_1.add(layers.MaxPooling2D((2, 2)))
model_1.add(layers.Conv2D(32, (3, 3), activation = 'relu'))
model_1.add(layers.MaxPooling2D((2, 2)))
model_1.add(layers.Flatten())
model_1.add(layers.Dense(512, activation = 'relu'))
model_1.add(layers.Dense(1, activation = 'sigmoid'))

In [ ]:
model_1.compile(loss = 'binary_crossentropy', optimizer = optimizers.RMSprop(lr = 1e-4), metrics = metrics)

In [ ]:
%%time

# Training the model
history_1 = model_1.fit_generator(
    train_generator,
    steps_per_epoch = 10,
    verbose = 0,
    epochs = 30,
    validation_data = dev_generator,
    validation_steps = 10)

In [ ]:
plot_history(history_1)

In [ ]:
# Predecting using test data
pred_1 = model_1.predict_generator(test_generator, steps=num_test_samples/batch_size).flatten()

In [ ]:
prediction_metrics(pred_1)

### Model 2
In this model we will experiment with pretrained model VGG16 downloaded from imagenet, we used [this](https://medium.com/unit8-machine-learning-publication/detecting-pneumonia-on-x-ray-images-covnets-and-transfer-learning-6d94b58c6657) articel as reference to implement this.

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(158, 158, 3))
flattened_output = layers.Flatten()(base_model.output)
final_output = layers.Dense(1, activation='sigmoid')(flattened_output)

model_2 = models.Model(inputs= base_model.input, outputs= final_output)

# to reduce trainable paramters set trainable to false for the first 20 layers (upto Flatten later)
for layer in model_2.layers[0:20]:
    layer.trainable = False

In [ ]:
model_2.compile(loss = 'binary_crossentropy', optimizer = optimizers.Adam(), metrics = metrics)

In [ ]:
%%time

# Training the model
history_2 = model_2.fit_generator(
    train_generator,
    steps_per_epoch = 10,
    verbose = 0,
    epochs = 10,
    validation_data = dev_generator,
    validation_steps = 10)

In [ ]:
plot_history(history_2)

In [ ]:
# Predecting using test data
pred_2 = model_2.predict_generator(test_generator, steps=num_test_samples/batch_size).flatten()

In [ ]:
prediction_metrics(pred_2)

### Model 3

In this model we tried to implement LeNet-5 classical NNN architecture using [this](https://medium.com/@mgazar/lenet-5-in-9-lines-of-code-using-keras-ac99294c8086) articel as reference to test how it would perform.

In [ ]:
model_3 = models.Sequential()
model_3.add(layers.Conv2D(6, (3, 3), activation='relu', input_shape=(158, 158, 3)))
model_3.add(layers.AveragePooling2D())
model_3.add(layers.Conv2D(16, (3, 3), activation='relu'))
model_3.add(layers.AveragePooling2D())
model_3.add(layers.Flatten())
model_3.add(layers.Dense(20, activation='relu'))
model_3.add(layers.Dense(84, activation='relu'))
model_3.add(layers.Dense(1, activation = 'sigmoid'))

In [ ]:
model_3.compile(loss = 'binary_crossentropy', optimizer = optimizers.Adam(), metrics = metrics)

In [ ]:
%%time

# Training the model
history_3 = model_3.fit_generator(
    train_generator,
    steps_per_epoch = 10,
    verbose = 0,
    epochs = 10,
    validation_data = dev_generator,
    validation_steps = 10)

In [ ]:
plot_history(history_3)

In [ ]:
# Predecting using test data
pred_3 = model_3.predict_generator(test_generator, steps=num_test_samples/batch_size).flatten()

In [ ]:
prediction_metrics(pred_3)

# 6 Conclusions  <a class="anchor" id="cas-conclusion"></a>

We have some very intersting results from our experiment, something that was common to all our models was that they performed unxpexpectedley worse on the test data set than the validation data set, this is unexpected since the validation dataset is not used in training the models. Overall we were able to create different models with acceptable performance.
Best perfoming model was Model2 which was construced using pre-trained network called VGG16. The reason Model2 is our winner model is because it have better accuracy and reduced time and computational power and better overall consistent high accuracy, the accuracy of the models change with each re-run due to random selection of the images.